# Определение нефтяных месторождений

## Описание проекта

Допустим, вы работаете в нефтедобывающей компании. Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## <a id='content'>Содержание проекта</a>
1. <a href='#step-1'>Загрузка и подготовка данных</a>
2. <a href='#step-2'>Обучение и проверка модели</a>
3. <a href='#step-3'>Подготовка к расчёту прибыли</a>
4. <a href='#step-4'>Расчёт прибыли и рисков</a>

# 1. <a id='step-1'>Загрузка и подготовка данных</a>

In [1]:
# Подключим нужные нам модули.
import pandas as pd
import numpy as np
from math import sqrt
from scipy.stats import percentileofscore
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Создадим датафреймы из файлов для каждого региона и выведем информацию о них, и первые 5 строк.
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(geo_data_0.info())
print(geo_data_1.info())
print(geo_data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

In [2]:
geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
geo_data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
geo_data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Пропусков и ошибок в данных не обнаружено, можем сразу перейти к обучению модели.

# 2. <a id='step-2'>Обучение и проверка модели</a>

Обучим и проверим модели отдельно для каждого региона. Для обучения будем использовать линейную регрессию, после чего проверим модель с помощью валидационной выборки. Модели для регионов будем строить по порядку.

In [5]:
# Создадим random_state.
state = np.random.RandomState(12345)

In [6]:
# Укажем признаки и целевой признак.
features_0 = geo_data_0.drop(['id', 'product'], axis=1)
target_0 = geo_data_0['product']

# Разбиваем на обучающую и валидационную выборки.
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=state)

# Обучим модель, после чего выведем предсказания для валидационной выборки.
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_0 = model_0.predict(features_valid_0)

# Рассчитаем средний запас сырья и RMSE модели, чтобы узнать, насколько велико отклонение.
print(f"Средний запас предсказанного сырья (в единицах продукта) в 'geo_data_0': {predictions_0.mean():.2f}")
print(f"RMSE модели в 'geo_data_0': {sqrt(mean_squared_error(predictions_0, target_valid_0)):.2f}\n")

Средний запас предсказанного сырья (в единицах продукта) в 'geo_data_0': 92.59
RMSE модели в 'geo_data_0': 37.58



И повторяем те же шаги для других регионов.

In [7]:
features_1 = geo_data_1.drop(['id', 'product'], axis=1)
target_1 = geo_data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=state)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_1 = model_1.predict(features_valid_1)
print(f"Средний запас предсказанного сырья (в единицах продукта) в 'geo_data_1': {predictions_1.mean():.2f}")
print(f"RMSE модели в 'geo_data_1': {sqrt(mean_squared_error(predictions_1, target_valid_1)):.2f}\n")

Средний запас предсказанного сырья (в единицах продукта) в 'geo_data_1': 68.77
RMSE модели в 'geo_data_1': 0.89



In [8]:
features_2 = geo_data_2.drop(['id', 'product'], axis=1)
target_2 = geo_data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=state)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_2 = model_2.predict(features_valid_2)
print(f"Средний запас предсказанного сырья (в единицах продукта) в 'geo_data_2': {predictions_2.mean():.2f}")
print(f"RMSE модели в 'geo_data_2': {sqrt(mean_squared_error(predictions_2, target_valid_2)):.2f}\n")

Средний запас предсказанного сырья (в единицах продукта) в 'geo_data_2': 95.09
RMSE модели в 'geo_data_2': 39.96



Как можно увидеть, во втором регионе хоть средний запаз и меньше, чем в первом и третьем регионах, но и RMSE тоже гораздо меньше, значит, качество предсказания модели выше.

# 3. <a id='step-3'>Подготовка к расчёту прибыли</a>

Для начала, сохраним данные, нужные нам для расчёта прибыли. Наш бюджет — 10 млрд. рублей, и цена за единицу продукта (1000 баррелей) 450 тысяч рублей. Всего мы будем разведывать 200 точек с наибольшим количеством продукта.

In [9]:
BUDGET = 10000000000
PRICE_PER_PRODUCT = 450000
POINTS_AMOUNT = 200

Найдём средние данные по количеству продукта по регионам и сравним с предсказанным значением.

In [10]:
print(f"По региону 1 — предсказанное значение: {geo_data_0['product'].mean():.2f} и реальное: {predictions_0.mean():.2f}.")
print(f"По региону 2 — предсказанное значение: {geo_data_1['product'].mean():.2f} и реальное: {predictions_1.mean():.2f}.")
print(f"По региону 3 — предсказанное значение: {geo_data_2['product'].mean():.2f} и реальное: {predictions_2.mean():.2f}.")

По региону 1 — предсказанное значение: 92.50 и реальное: 92.59.
По региону 2 — предсказанное значение: 68.83 и реальное: 68.77.
По региону 3 — предсказанное значение: 95.00 и реальное: 95.09.


Как видно, предсказанные значения не сильно отличаются от реальных.

Расчитаем минимум продукта в одной точке для безубыточной добычи.

In [11]:
print(f"{BUDGET / PRICE_PER_PRODUCT / POINTS_AMOUNT:.2f}")

111.11


Как можно заметить, средние значения по регионам получаются меньше, чем рассчитанное минимальное количество продукта в точке. Поэтому мы стараемся найти точки, где количество продукта будет превышать это значение.

Напишем функцию для подсчёта прибыли от региона.

In [12]:
def count_profit(target, predictions, count):  
    '''
    Считает прибыль, умножая сумму из selected на 
    price_per_product и вычитая бюджет. selected 
    берётся из target, соответствующих индексам predictions_sorted.
    '''
    predictions = pd.Series(predictions)
    target = pd.Series(target)
    
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:count]
    
    return PRICE_PER_PRODUCT * selected.sum() - BUDGET

Теперь, приступим к расчёту прибыли и рисков.

# 4. <a id='step-4'>Расчёт прибыли и рисков</a>

Воспользуемся методом Bootstrap для создания распределения величин из подвыборок. Возьмём тысячу подвыборок.

In [13]:
# Создаём пустой список, куда мы будем записывать данные с бутстрепа.
values_0 = []

target_valid_0 = pd.Series(target_valid_0).reset_index(drop=True)
predictions_0 = pd.Series(predictions_0)

# Применим бутстреп.
for i in range(1000):
    # Берём 500 случайных значений и находим соответствующие по индексу значения в другой выборке.
    target_subsample_0 = target_valid_0.sample(n=500, replace=True, random_state=state) 
    pred_subsample_0 = predictions_0[target_subsample_0.index]
    
    # Применяем функцию расчёта прибыли.
    values_0.append(count_profit(target_subsample_0, pred_subsample_0, POINTS_AMOUNT))

# Переводим values_0 в Series.
values_0 = pd.Series(values_0)

In [14]:
# Находим нижний и верхний квантили для 95-процентного доверительного интервала.
lower_0 = values_0.quantile(0.025)
upper_0 = values_0.quantile(0.975)

# Находим среднее по values_0.
mean_0 = values_0.mean()

# И вероятность убытков.
prob_of_loss_0 = percentileofscore(values_0, 0)
print(f"Регион 1 с вероятностью 95% заработает от {lower_0:.0f} рублей до {upper_0:.0f} рублей (ср: {mean_0:.0f} рублей).")
print(f"Убыток будет в {prob_of_loss_0:.1f}% случаев.")

Регион 1 с вероятностью 95% заработает от -76187814 рублей до 957846532 рублей (ср: 423897238 рублей).
Убыток будет в 4.8% случаев.


Повторяем те же шаги для остальных регионов.

In [15]:
values_1 = []
target_valid_1 = pd.Series(target_valid_1).reset_index(drop=True)
predictions_1 = pd.Series(predictions_1)

for i in range(1000):
    target_subsample_1 = target_valid_1.sample(n=500, replace=True, random_state=state)
    pred_subsample_1 = predictions_1[target_subsample_1.index]
    
    values_1.append(count_profit(target_subsample_1, pred_subsample_1, POINTS_AMOUNT))
       
values_1 = pd.Series(values_1)

In [16]:
lower_1 = values_1.quantile(0.025)
upper_1 = values_1.quantile(0.975)
mean_1 = values_1.mean()
prob_of_loss_1 = percentileofscore(values_1, 0)
print(f"Регион 1 с вероятностью 95% заработает от {lower_1:.0f} рублей до {upper_1:.0f} рублей (ср: {mean_1:.0f} рублей).")
print(f"Убыток будет в {prob_of_loss_1:.1f}% случаев.")

Регион 1 с вероятностью 95% заработает от 108066895 рублей до 928574439 рублей (ср: 513256699 рублей).
Убыток будет в 0.6% случаев.


In [17]:
values_2 = []
target_valid_2 = pd.Series(target_valid_2).reset_index(drop=True)
predictions_2 = pd.Series(predictions_2)
                          
for i in range(1000):                      
    target_subsample_2 = target_valid_2.sample(n=500, replace=True, random_state=state)
    pred_subsample_2 = predictions_2[target_subsample_2.index]
    
    values_2.append(count_profit(target_subsample_2, pred_subsample_2, POINTS_AMOUNT))
       
values_2 = pd.Series(values_2)

In [18]:
lower_2 = values_2.quantile(0.025)
upper_2 = values_2.quantile(0.975)
mean_2 = values_2.mean()
prob_of_loss_2 = percentileofscore(values_2, 0)
print(f"Регион 1 с вероятностью 95% заработает от {lower_2:.0f} рублей до {upper_2:.0f} рублей (ср: {mean_2:.0f} рублей).")
print(f"Убыток будет в {prob_of_loss_2:.1f}% случаев.")

Регион 1 с вероятностью 95% заработает от -142800630 рублей до 893380566 рублей (ср: 381120360 рублей).
Убыток будет в 7.4% случаев.


Можно сделать вывод, что регион 2 будет лучшим выбором по двум причинам. Первая, и самая главная: риск убытков всего 0.6%, тогда как в остальных двух регионах он 4.8% и 7.4%. И средняя прибыль в регионе 2 больше, чем в остальных двух. Вывод: добычу лучше всего проводить в регионе 2.

## <a href='#content'>Возврат к содержанию</a>